# Resumo

1. Vamos coletar diversas descrições de notícias do ** Jornal Estado de Minas ** via ** RSS ** com o pacote ** feedparser **.

2. Iremos importar todo o conteúdo extraído para o MongoDB

3. Vamos contar a frequência das palavras dessas descrições.

OBS: Vale ressaltar que esta análise não está considerando **STOPWORDS** e **STEMMING**.


# 1. Coleta

### Importar os pacotes a serem utilizados

In [77]:
from pymongo import MongoClient
import feedparser
import string
import html # Convert HTML accents to text
import unidecode # Remove accents
import re

### Conexão com o Banco

In [78]:
client = MongoClient('localhost', 27017)
db = client.aula
news = db.news

### Leitura do RSS

In [80]:
# 1. Parseamos o RSS diante da URL do RSS do EM
em_rss = feedparser.parse('http://www.em.com.br/rss/noticia/gerais/rss.xml')

# 2. Vamos iterar por todos os registros e extrair o sumário de todas as notícias. (list comprehesion)
summary_list = [news.summary for news in em_rss.entries]

# 3. Verificamos o número de sumários extraídos
print('Número de sumários extraídos: {total}'.format(total=len(summary_list)))

# 4. Vamos iterar por cada registro, remover as pontuações e inserir no banco.
for description in summary_list:
    cleaned_description = html.unescape(description) # Remove HTML characters
    cleaned_description = unidecode.unidecode(cleaned_description).translate(string.punctuation) # Remove accents
    cleaned_description = re.sub(r"[^a-zA-Z]+", " ", cleaned_description) # Remove SPECIAL Characters
    news.insert_one({'description': cleaned_description})

Número de sumários extraídos: 20


In [81]:
# Checamos se foram inseridos os registros no banco
print(news.count())
print(news.find_one())

40
{'_id': ObjectId('593f4218e6f8e7e66d1a4ad0'), 'description': 'Agentes da Policia Civil da Delegacia de Conquista no Triangulo Mineiro a quilometros da capital prederam tres envolvidos num esquema de furto de equipamentos tecnologicos de maquinas agricolas Os aparelhos que sao receptores de sinais de satelite usados para navegacao de colheitadeiras entre outras que em media custam R mil eram vendidas por R mil pelos criminosos O material recuperado policiais foi avaliado em mais de R mil A investigacao foi iniciada em janeiro deste ano depois do registro de furtos dos equipamentos modelos Star Fire em propriedades na zona rural de Conquista '}


# 2. Processamento

### Contagem da frequência de palavras

### Pythonic Way

In [82]:
words = []

# Pegamos todos os registros
description_list = news.find()

# Quebramos todas as frases em palavras.
for word in description_list:
    words.extend(word["description"].split())
    
print(words)
print('Número de ocorrências: {num}'.format(num=len(words)))

['Agentes', 'da', 'Policia', 'Civil', 'da', 'Delegacia', 'de', 'Conquista', 'no', 'Triangulo', 'Mineiro', 'a', 'quilometros', 'da', 'capital', 'prederam', 'tres', 'envolvidos', 'num', 'esquema', 'de', 'furto', 'de', 'equipamentos', 'tecnologicos', 'de', 'maquinas', 'agricolas', 'Os', 'aparelhos', 'que', 'sao', 'receptores', 'de', 'sinais', 'de', 'satelite', 'usados', 'para', 'navegacao', 'de', 'colheitadeiras', 'entre', 'outras', 'que', 'em', 'media', 'custam', 'R', 'mil', 'eram', 'vendidas', 'por', 'R', 'mil', 'pelos', 'criminosos', 'O', 'material', 'recuperado', 'policiais', 'foi', 'avaliado', 'em', 'mais', 'de', 'R', 'mil', 'A', 'investigacao', 'foi', 'iniciada', 'em', 'janeiro', 'deste', 'ano', 'depois', 'do', 'registro', 'de', 'furtos', 'dos', 'equipamentos', 'modelos', 'Star', 'Fire', 'em', 'propriedades', 'na', 'zona', 'rural', 'de', 'Conquista', 'Minas', 'Gerais', 'registrou', 'mais', 'uma', 'morte', 'por', 'suspeita', 'de', 'febre', 'chikungunya', 'no', 'periodo', 'de', 'uma',

###  Mongo Way

In [65]:
from bson.code import Code

map = Code(""" 
            function () {
              this.description.trim().split(/\s+/).forEach((z) => { emit(z, 1) });
            };
           """)

reduce = Code("""
                function(key, values) {
                    return Array.sum(values)
                }
              """)

result = news.map_reduce(map, reduce, "wordcount")

### Exibição dos resultados.

In [83]:
# Python Way
word_freq = {}

for word in words:
    # Se a palavra não estiver no dicionário de frequencia, então adicione ela e a frequência.
    if not word in word_freq.keys():
        word_freq.update({word: words.count(word)})

word_freq

{'A': 24,
 'Acidente': 1,
 'Acima': 1,
 'Agentes': 2,
 'Agora': 2,
 'Aguda': 3,
 'Aleixo': 1,
 'Alem': 2,
 'Alex': 1,
 'Altas': 1,
 'Alto': 1,
 'Antonio': 2,
 'Araxa': 2,
 'As': 4,
 'Augusto': 2,
 'Avenida': 4,
 'B': 1,
 'BH': 1,
 'BR': 2,
 'Bairro': 9,
 'Balanco': 2,
 'Barreiro': 2,
 'Barro': 2,
 'Batista': 2,
 'Belo': 12,
 'Belvedere': 1,
 'Bombeiros': 1,
 'Botanico': 1,
 'Branco': 1,
 'Brinks': 1,
 'Caicara': 2,
 'Caique': 1,
 'Campo': 2,
 'Campos': 1,
 'Carmo': 2,
 'Carrapatoso': 1,
 'Castro': 2,
 'Catas': 1,
 'Celio': 2,
 'Central': 4,
 'Centro': 6,
 'Civil': 15,
 'Claros': 2,
 'Cobalt': 2,
 'Comdec': 4,
 'Complexo': 3,
 'Conforme': 1,
 'Conquista': 4,
 'Conselheiro': 1,
 'Constricao': 3,
 'Contorno': 2,
 'Contra': 1,
 'Coordenadoria': 4,
 'Corpo': 1,
 'Couto': 2,
 'Crimes': 1,
 'Criminosos': 2,
 'Cullinan': 1,
 'Dalmazio': 2,
 'Das': 1,
 'De': 4,
 'Defesa': 4,
 'Delegacia': 2,
 'Denis': 2,
 'Deoesp': 1,
 'Departamento': 1,
 'Desconfiado': 1,
 'Dia': 2,
 'Do': 2,
 'Dr': 2,
 'Duas'

In [73]:
# MongoDB Way
for words in db.wordcount.find():
    print(words)

{'_id': 'A', 'value': 13.0}
{'_id': 'Acidente', 'value': 1.0}
{'_id': 'Acima', 'value': 1.0}
{'_id': 'Agentes', 'value': 1.0}
{'_id': 'Agora', 'value': 1.0}
{'_id': 'Aguda', 'value': 2.0}
{'_id': 'Alem', 'value': 1.0}
{'_id': 'Alex', 'value': 1.0}
{'_id': 'Altas', 'value': 1.0}
{'_id': 'Antonio', 'value': 1.0}
{'_id': 'As', 'value': 2.0}
{'_id': 'Augusto', 'value': 1.0}
{'_id': 'Avenida', 'value': 2.0}
{'_id': 'B', 'value': 1.0}
{'_id': 'BH', 'value': 1.0}
{'_id': 'BR', 'value': 1.0}
{'_id': 'Bairro', 'value': 5.0}
{'_id': 'Balanco', 'value': 1.0}
{'_id': 'Barreiro', 'value': 1.0}
{'_id': 'Barro', 'value': 1.0}
{'_id': 'Batista', 'value': 1.0}
{'_id': 'Belo', 'value': 4.0}
{'_id': 'Bombeiros', 'value': 1.0}
{'_id': 'Branco', 'value': 1.0}
{'_id': 'Caicara', 'value': 1.0}
{'_id': 'Caique', 'value': 1.0}
{'_id': 'Campo', 'value': 1.0}
{'_id': 'Campos', 'value': 1.0}
{'_id': 'Carmo', 'value': 1.0}
{'_id': 'Carrapatoso', 'value': 1.0}
{'_id': 'Castro', 'value': 1.0}
{'_id': 'Catas', 'value